# Import Bibliotheken

In [1]:
# Festlegung des Device
import platform

# Laden der Daten
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

# Operationen mit Texten
import torchtext
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
import spacy

# Modell
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import optim

# Evaluierung
import torchmetrics

# Auswertung der Evaluation
import pandas as pd
import altair as alt
import numpy as np

from sklearn import preprocessing

### Package Versionen

In [2]:
for package in [torchtext,torch, torchmetrics, spacy, alt, pd]:
    print(f'{package.__name__}: {package.__version__}')

torchtext: 0.13.1
torch: 1.12.1
torchmetrics: 0.9.3
spacy: 3.4.2
altair: 4.2.0
pandas: 1.4.3


# Device Auswahl

In [3]:
device = torch.device(
    "cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

# Daten

## Dataset

### Erstellung Dataset

In [4]:
comment_df = pd.read_excel('data/clean/Google_Rezensionen.xlsx')
comment_df

,Unnamed: 0,Tankstellenname,Datum,Bewertung,Kommentar,Kategorien
0,4,TS Schleswig MC,2022-09-30 06:46:42,5,super nettes personal gutes frühstück mit groß...,"Personal,Shop,Bistro"
1,5,TS Prisdorf,2022-09-29 17:22:44,5,sehr höfliche bedienstete,Personal
2,6,TS Wanderup,2022-09-29 15:38:02,5,immer gerne dort nettes und zuvorkommendes per...,"Personal,Waschanlage"
3,10,TS Handewitt,2022-09-28 17:05:28,5,nettes freundliches personal,Personal
4,12,TS Bremen,2022-09-28 04:24:34,4,normale tankstelle die aber in der tank app ni...,"Pricing,DigitalFueling"
...,...,...,...,...,...,...
3992,11072,TS Jübek,2015-04-07 17:45:02,5,dies ist einfach die beste tankstelle die ich ...,"Kraftstoffauswahl,Waschanlage"
3993,11077,TS Jübek,2014-07-16 07:08:37,5,sehr gut günstig guter shop und einige andere ...,"Pricing,Shop,Kraftstoffauswahl,Waschanlage,Sta..."
3994,11079,TS Neustadt am Rüb.,2013-09-21 15:26:28,5,öffnungszeiten mo so 06 00 22 00 uhr,Öffnungszeiten
3995,11080,TS Handewitt(SP),2012-11-05 11:16:00,5,genug platz fuer wohnwagen rechts beim autogas...,"Personal,Kraftstoffauswahl"


In [5]:
class TeamGoogleBewertungenDataSet(Dataset):

    def __init__(self, data):
        self.data = data
    
    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        text = comment_df.iloc[index]['Kommentar']
        category = comment_df.iloc[index]['Kategorien']
        return text, category

### Bestimmung Test- und Traningsdaten

In [6]:
# Bestimmung der Länge der Test- und Traningsdaten
dataset = TeamGoogleBewertungenDataSet(comment_df)
len_train = round(len(dataset)* 0.7)
len_test = round(len(dataset) * 0.3)
assert len(comment_df) == len_train + len_test

In [7]:
# Zufällige Aufsplittung der Anzahl von Test- und Trainingsdaten
train_set, test_set = torch.utils.data.random_split(dataset, [len_train, len_test]) 

In [8]:
len(train_set)

2798

In [9]:

len(test_set)

1199

## Dataloader

### Tokenisierung

In [10]:
tokenizer = get_tokenizer(
    'spacy', 
    language='de_core_news_lg')

### Erstellung Vokabular

In [11]:
def yield_tokens(data_iter):
    for text, _ in data_iter:
        yield tokenizer(text)

In [12]:
tokens = yield_tokens(train_set)

In [13]:
next(iter(tokens))

['tolle', 'und', 'moderne', 'tankstelle']

In [14]:
vocab = build_vocab_from_iterator(tokens, min_freq=2, specials=["<unk>"])

vocab.set_default_index(vocab["<unk>"])

In [15]:
len(vocab)

2409

In [16]:
vocab.lookup_token(0)

'<unk>'

### Encoding

In [17]:
#onehot = torch.zeros(1,len(vocab))

In [18]:
# pos = vocab(['nette', 'Bedienung', 'sauber'])
# pos

In [19]:
def encode(text, vocab):
    tokens = tokenizer(text)
    onehot = torch.zeros(1,len(vocab))
    onehot[:,vocab(tokens)] = 1
    return onehot

### Multilabel Encoding

In [20]:
import torch
import torch.nn

In [21]:
df = comment_df.explode('Kategorien')
df

,Unnamed: 0,Tankstellenname,Datum,Bewertung,Kommentar,Kategorien
0,4,TS Schleswig MC,2022-09-30 06:46:42,5,super nettes personal gutes frühstück mit groß...,"Personal,Shop,Bistro"
1,5,TS Prisdorf,2022-09-29 17:22:44,5,sehr höfliche bedienstete,Personal
2,6,TS Wanderup,2022-09-29 15:38:02,5,immer gerne dort nettes und zuvorkommendes per...,"Personal,Waschanlage"
3,10,TS Handewitt,2022-09-28 17:05:28,5,nettes freundliches personal,Personal
4,12,TS Bremen,2022-09-28 04:24:34,4,normale tankstelle die aber in der tank app ni...,"Pricing,DigitalFueling"
...,...,...,...,...,...,...
3992,11072,TS Jübek,2015-04-07 17:45:02,5,dies ist einfach die beste tankstelle die ich ...,"Kraftstoffauswahl,Waschanlage"
3993,11077,TS Jübek,2014-07-16 07:08:37,5,sehr gut günstig guter shop und einige andere ...,"Pricing,Shop,Kraftstoffauswahl,Waschanlage,Sta..."
3994,11079,TS Neustadt am Rüb.,2013-09-21 15:26:28,5,öffnungszeiten mo so 06 00 22 00 uhr,Öffnungszeiten
3995,11080,TS Handewitt(SP),2012-11-05 11:16:00,5,genug platz fuer wohnwagen rechts beim autogas...,"Personal,Kraftstoffauswahl"


#### Kategorien splitten

In [22]:
einzelLabels = comment_df['Kategorien'].str.split(',')
label_list_export = einzelLabels.to_list()
label_list_export

[['Personal', 'Shop', 'Bistro'],
 ['Personal'],
 ['Personal', 'Waschanlage'],
 ['Personal'],
 ['Pricing', 'DigitalFueling'],
 ['Erscheinungsbild', 'Personal'],
 ['Personal', 'Pricing'],
 ['Personal', 'Pricing', 'Kraftstoffauswahl'],
 ['Personal', 'Öffnungszeiten'],
 ['Personal', 'SB-Waschboxen', 'Waschanlage'],
 ['Shop'],
 ['Erscheinungsbild', 'Personal'],
 ['Personal', 'Waschanlage'],
 ['Personal', 'Pricing'],
 ['Personal', 'Pricing'],
 ['Bistro'],
 ['Personal', 'Pricing', 'Shop', 'Waschanlage'],
 ['Personal'],
 ['Personal'],
 ['Bistro'],
 ['Erscheinungsbild'],
 ['Personal', 'Pricing'],
 ['Personal'],
 ['Personal'],
 ['Personal', 'Shop'],
 ['Erscheinungsbild'],
 ['Personal', 'Bistro'],
 ['Erscheinungsbild', 'Nacht-/Tankautomat', 'Waschanlage', 'Staubsauger'],
 ['Pricing'],
 ['Personal'],
 ['Erscheinungsbild', 'Personal'],
 ['Personal'],
 ['Bistro'],
 ['Pricing', 'Kraftstoffauswahl'],
 ['Pricing'],
 ['Nacht-/Tankautomat'],
 ['Shop', 'Bistro'],
 ['Erscheinungsbild', 'Personal', 'Sanitär

##### In der nachfolgenden Zelle wird aus der verschachtelten Liste eine "flache Liste"

In [23]:
flat_list = []
for sublist in label_list_export:
    for item in sublist:
        flat_list.append(item)

flat_list

['Personal',
 'Shop',
 'Bistro',
 'Personal',
 'Personal',
 'Waschanlage',
 'Personal',
 'Pricing',
 'DigitalFueling',
 'Erscheinungsbild',
 'Personal',
 'Personal',
 'Pricing',
 'Personal',
 'Pricing',
 'Kraftstoffauswahl',
 'Personal',
 'Öffnungszeiten',
 'Personal',
 'SB-Waschboxen',
 'Waschanlage',
 'Shop',
 'Erscheinungsbild',
 'Personal',
 'Personal',
 'Waschanlage',
 'Personal',
 'Pricing',
 'Personal',
 'Pricing',
 'Bistro',
 'Personal',
 'Pricing',
 'Shop',
 'Waschanlage',
 'Personal',
 'Personal',
 'Bistro',
 'Erscheinungsbild',
 'Personal',
 'Pricing',
 'Personal',
 'Personal',
 'Personal',
 'Shop',
 'Erscheinungsbild',
 'Personal',
 'Bistro',
 'Erscheinungsbild',
 'Nacht-/Tankautomat',
 'Waschanlage',
 'Staubsauger',
 'Pricing',
 'Personal',
 'Erscheinungsbild',
 'Personal',
 'Personal',
 'Bistro',
 'Pricing',
 'Kraftstoffauswahl',
 'Pricing',
 'Nacht-/Tankautomat',
 'Shop',
 'Bistro',
 'Erscheinungsbild',
 'Personal',
 'Sanitär',
 'Bistro',
 'Pricing',
 'Verkehrsanbindung'

### Grafische Darstellung von der Anzahl der Kategorien

In [24]:
count_list = ([ [l, flat_list.count(l)] for l in set(flat_list)])
count_list

[['Kraftstoffauswahl', 280],
 ['Pricing', 1000],
 ['Verkehrsanbindung', 111],
 ['Kartenakzeptanzen', 145],
 ['DigitalFueling', 30],
 ['Waschanlage', 462],
 ['Nacht-/Tankautomat', 95],
 ['Staubsauger', 32],
 ['Werkstatt', 45],
 ['E-Fuels', 1],
 ['Tankpool', 3],
 ['Personal', 2790],
 ['WLAN', 3],
 ['Luft', 86],
 ['SB-Waschboxen', 45],
 ['Öffnungszeiten', 129],
 ['E-Mobilität', 7],
 ['Shop', 554],
 ['AdBlue', 37],
 ['Paketservice', 32],
 ['Bistro', 716],
 ['Zapfsäulen', 45],
 ['Erscheinungsbild', 534],
 ['Sanitär', 138]]

In [25]:
count_df = pd.DataFrame(count_list)
count_df.columns= ['Kategorie', 'Anzahl']
count_df

,Kategorie,Anzahl
0,Kraftstoffauswahl,280
1,Pricing,1000
2,Verkehrsanbindung,111
3,Kartenakzeptanzen,145
4,DigitalFueling,30
5,Waschanlage,462
6,Nacht-/Tankautomat,95
7,Staubsauger,32
8,Werkstatt,45
9,E-Fuels,1


In [26]:
kategorien_anzahl_bar = alt.Chart(count_df).mark_bar(color='#EC6500').encode(
    x = alt.X('Anzahl', title='Anzahl Vorkomnisse'),
    y = alt.Y('Kategorie', title='Kategorie', sort='-x')
).properties(
    title='Kategorieklassen'
)
kategorien_anzahl_text = kategorien_anzahl_bar.mark_text(
    align='left',
    baseline= 'middle',
    dx=3
).encode(
    text='Anzahl:Q'
)

kategorien_anzahl_chart = (kategorien_anzahl_bar + kategorien_anzahl_text)
kategorien_anzahl_chart.properties(width=550)

alt.LayerChart(...)

##### Doppelte Werte werden entfernt, so dass jeder Eintrag nur einmalig vorhanden ist. Dies erreichen wir mit set()

In [27]:
flat_set = set(flat_list)
flat_set

{'AdBlue',
 'Bistro',
 'DigitalFueling',
 'E-Fuels',
 'E-Mobilität',
 'Erscheinungsbild',
 'Kartenakzeptanzen',
 'Kraftstoffauswahl',
 'Luft',
 'Nacht-/Tankautomat',
 'Paketservice',
 'Personal',
 'Pricing',
 'SB-Waschboxen',
 'Sanitär',
 'Shop',
 'Staubsauger',
 'Tankpool',
 'Verkehrsanbindung',
 'WLAN',
 'Waschanlage',
 'Werkstatt',
 'Zapfsäulen',
 'Öffnungszeiten'}

In [28]:
unflat = [[x] for x in flat_set]

In [29]:
label_vocab = build_vocab_from_iterator(unflat, min_freq=1, specials=["<unk>"])

label_vocab.set_default_index(vocab["<unk>"])

In [30]:
label_vocab.lookup_token(3)

'DigitalFueling'

#### Dictionary erstellen

In [31]:
label_dict = {label:i for i, label in enumerate(flat_set)}
label_dict

{'Kraftstoffauswahl': 0,
 'Pricing': 1,
 'Verkehrsanbindung': 2,
 'Kartenakzeptanzen': 3,
 'DigitalFueling': 4,
 'Waschanlage': 5,
 'Nacht-/Tankautomat': 6,
 'Staubsauger': 7,
 'Werkstatt': 8,
 'E-Fuels': 9,
 'Tankpool': 10,
 'Personal': 11,
 'WLAN': 12,
 'Luft': 13,
 'SB-Waschboxen': 14,
 'Öffnungszeiten': 15,
 'E-Mobilität': 16,
 'Shop': 17,
 'AdBlue': 18,
 'Paketservice': 19,
 'Bistro': 20,
 'Zapfsäulen': 21,
 'Erscheinungsbild': 22,
 'Sanitär': 23}

#### Erstellung Dataloader

In [39]:
def collate_batch(batch):
    label_list, text_list = [], []
 
    for (_text,_label) in batch:
    
        # Vorverarbeitung der Label
        # label_list.append(label_pipeline(_label))
        processed_labels = encode(_label, label_vocab)

        label_list.append(processed_labels)

        # Vorverarbeitung der Texte
        processed_text = encode(_text, vocab)
        
        # Zusammenführen sämtlicher Textrepräsentationen in einer Liste
        text_list.append(processed_text)
 
    # Zusammenführen aller Label in einem Tensor
    labels = torch.cat(label_list, dim=0)
    
    # Verbinden der Tensoren in text_list zu einem Tensor
    texts = torch.cat(text_list, dim = 0)

    # Ausgabe der Texte und der Label
    return texts.to(device), labels.to(device)

In [71]:
train_loader = DataLoader(train_set, batch_size=64,shuffle=True,collate_fn=collate_batch,num_workers=0)

## Architektur und Training

### Architektur

In [41]:
class LinearTextClassificationModel(nn.Module):

    def __init__(self, vocab_size, num_class):
        super(LinearTextClassificationModel, self).__init__()
        self.fc1 = nn.Linear(vocab_size, 200)
        self.fc2 = nn.Linear(200, num_class)

    def forward(self, x):
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        output = F.softmax(x, dim=1)
        return output

In [42]:
vocab_size = len(vocab)
num_class = 25
model = LinearTextClassificationModel(vocab_size, num_class).to(device)

### Training

In [43]:
# Hyperparameter

## Festlegung Lernrate
learning_rate = 0.05

## Initialisierung Fehlerfunktion
loss_fn = nn.BCEWithLogitsLoss()

## Initialisierung Optimizer
optimizer = torch.optim.SGD(model.parameters(),lr = learning_rate, momentum=0.9)

## Definition der Epochen
num_epochs = 50

In [44]:
train_accuracy = torchmetrics.Accuracy(subset_accuracy=True,threshold=0.6).to(device)

loss_hist = {}
accuracy_hist = {}

In [45]:
for (text, label) in train_loader:
    #print(label[[0][0]])
    #print(text)
    labelnew = label.type(torch.LongTensor)
    print(labelnew[[0][0]])

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0])
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0])
tensor([1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0])
tensor([1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
        0])
tensor([1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0])
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0])
tensor([1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1,
        0])
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0])
tensor([1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0])
tensor([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0])
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,


In [46]:
# Training
model.train()

for epoch in range(num_epochs):

    # Dokumentation Loss -> Erkennung ob Netz konvergiert
    running_loss = 0.0
    num_batches = 0

    for (text, label) in train_loader:
        num_batches += 1
        pred = model(text)

        # Der Fehler wird berechnet
        loss = loss_fn(pred, label)
        # Der Fehler wird über das Netz zurückpropagiert
        loss.backward()
        # Die Gewichte werden angepasst
        optimizer.step()
        # Gradienten zurücksetzen
        optimizer.zero_grad()
        ## Bestimmung der Accuracy für den Batch
        train_accuracy(pred, label.type(torch.LongTensor))
        
        # running loss
        running_loss +=loss.item()
 
    loss_hist[epoch] = running_loss/num_batches
 
    batch_train_accuracy = train_accuracy.compute()
    print(f"Training Accuracy for epoch {epoch}:{batch_train_accuracy}")
    accuracy_hist[epoch] = batch_train_accuracy.cpu().item()
    train_accuracy.reset()

Training Accuracy for epoch 0:0.0
Training Accuracy for epoch 1:0.0
Training Accuracy for epoch 2:0.0
Training Accuracy for epoch 3:0.0
Training Accuracy for epoch 4:0.0
Training Accuracy for epoch 5:0.0
Training Accuracy for epoch 6:0.0
Training Accuracy for epoch 7:0.0
Training Accuracy for epoch 8:0.0
Training Accuracy for epoch 9:0.0
Training Accuracy for epoch 10:0.0
Training Accuracy for epoch 11:0.0
Training Accuracy for epoch 12:0.0
Training Accuracy for epoch 13:0.0
Training Accuracy for epoch 14:0.0
Training Accuracy for epoch 15:0.0
Training Accuracy for epoch 16:0.0
Training Accuracy for epoch 17:0.0
Training Accuracy for epoch 18:0.0
Training Accuracy for epoch 19:0.0
Training Accuracy for epoch 20:0.0
Training Accuracy for epoch 21:0.0
Training Accuracy for epoch 22:0.0
Training Accuracy for epoch 23:0.0007147963042370975
Training Accuracy for epoch 24:0.006433166563510895
Training Accuracy for epoch 25:0.02573266625404358
Training Accuracy for epoch 26:0.0829163715243339

### Auswertung Fehlerentwicklung

In [47]:
loss_df = pd.DataFrame.from_dict(loss_hist, orient= 'index').reset_index()

loss_df.columns = ['Epoch', 'Loss']
loss_df.head()

,Epoch,Loss
0,0,0.709578
1,1,0.709536
2,2,0.709474
3,3,0.709419
4,4,0.709355


In [48]:
train_chart = alt.Chart(loss_df).mark_line().encode(
    x=alt.X('Epoch', title = 'Anzahl Epochen'),
    y=alt.Y('Loss', title = 'Mittlerer Fehler')
)
# glue('train-loss-team', train_chart,display=True)
train_chart

alt.Chart(...)

### Auswertung Genauigkeit

In [49]:
accuracy_df = pd.DataFrame.from_dict(accuracy_hist, orient = 'index').reset_index()
accuracy_df.columns = ['Epoch', 'Accuracy']
accuracy_df.head()

,Epoch,Accuracy
0,0,0.0
1,1,0.0
2,2,0.0
3,3,0.0
4,4,0.0


In [50]:
accuracy_chart = alt.Chart(accuracy_df).mark_line().encode(
    x=alt.X('Epoch',title = 'Anzahl Epochen'),
    y=alt.Y('Accuracy', title = 'Genauigkeit')
)
accuracy_chart

alt.Chart(...)

## Evaluierung

In [51]:
model.eval()

LinearTextClassificationModel(
  (fc1): Linear(in_features=2409, out_features=200, bias=True)
  (fc2): Linear(in_features=200, out_features=25, bias=True)
)

### Gesamtevaluierung

In [52]:
test_loader = DataLoader(test_set, batch_size= 64, shuffle=True,collate_fn=collate_batch)

In [53]:
valid_accuracy = torchmetrics.Accuracy(subset_accuracy=True,threshold=0.6).to(device)

with torch.no_grad():
    for (text, label) in test_loader:
        # Vorhersage wird für das Model erzeugt
        pred = model(text)
        valid_accuracy.update(pred, label.type(torch.LongTensor))
    total_valid_accuracy = valid_accuracy.compute()

In [54]:
total_valid_accuracy

tensor(0.3094)

### Evaluierung nach Klassen

In [55]:
# Evaluieren
valid_accuracy = torchmetrics.Accuracy(
    average=None,
    num_classes=25
).to(device)

with torch.no_grad():
    for(text, label) in test_loader:
        # Die Vorhersage wird für das Model erzeugt
        pred = model(text)
        valid_accuracy.update(pred, label.type(torch.LongTensor))
    total_valid_accuracy = valid_accuracy.compute()
    total_valid_accuracy

In [56]:
total_valid_accuracy

tensor([0.5129, 0.9933, 0.8224, 0.9942,    nan, 0.9975, 0.8590, 0.9616, 0.9266,
        0.9741, 0.9725, 0.9942, 0.6881, 0.7331, 0.9908, 0.9650, 0.8707, 0.9908,
           nan, 0.9783, 0.9983, 0.8941, 0.9892, 0.9892, 0.9691])

### Evaluierung nach der Anzahl der Epochen

In [57]:
train_loader = DataLoader(train_set, batch_size=64, shuffle=True, collate_fn=collate_batch)

In [58]:
test_loader = DataLoader(test_set, batch_size=64, shuffle=True, collate_fn=collate_batch)

In [59]:
vocab_size = len(vocab)
num_class = 25
model = LinearTextClassificationModel(vocab_size, num_class).to(device)

In [60]:
# Hyperparameter

## Festlegung Lernrate
learning_rate = 0.05

## Initialisierung Fehlerfunktion
loss_fn = nn.BCEWithLogitsLoss()

## Initialisierung Optimizer
optimizer = torch.optim.SGD(model.parameters(),lr = learning_rate, momentum=0.9)

## Definition der Epochen
num_epochs = 50

In [61]:
# Evaluierung wurd vorbereitet
train_accuracy = torchmetrics.Accuracy(subset_accuracy=True,threshold=0.6).to(device)
valid_accuracy = torchmetrics.Accuracy(subset_accuracy=True,threshold=0.6).to(device)

train_accuracy_hist = {}
train_loss_hist = {}

valid_accuracy_hist = {}
valid_loss_hist = {}

In [62]:
for epoch in range(num_epochs):

    # Loss wird dokumentiert, sodass zu erkenne ist, ob das Netz konvergiert
    num_train_batches = 0
    num_eval_batches = 0
    train_running_loss = 0.0
    eval_running_loss = 0.0

    model.train()
    for (text, label) in train_loader:
        num_train_batches += 1
        # Die Vorhersage wird für das Model erzeugt
        pred = model(text)
        # Der Fehler wird berechnet
        # Als Ausgabe wird ein Tensor zurückgegeben
        loss = loss_fn(pred, label)
        # Der Fehler wird über das Netz zurükpropagiert
        loss.backward()
        # Die Gewichte werden angepasst
        optimizer.step()
        # Gradienten zurücksetzen
        optimizer.zero_grad()
        # Aktualisierung des States der train_accurracy
        train_accuracy.update(pred, label.type(torch.LongTensor))
        # running loss
        train_running_loss += loss.item()

    train_loss_hist[epoch] = train_running_loss / num_train_batches

    # Berechnung der Trainingsgenauigkeit für die Epoche
    total_train_accuracy = train_accuracy.compute()
    # Hinzufügen der Trainingsgenauigkeit zu dem Dictionary
    train_accuracy_hist[epoch] = total_train_accuracy.cpu().numpy()

    # Validierung
    model.eval()
    with torch.no_grad():
        for (text, label) in test_loader:
            num_eval_batches += 1
            # Die Vorhersage wird für das Model erzeugt
            pred = model(text)
            # Test loss
            eval_loss = loss_fn(pred, label)
            # Running loss
            eval_running_loss += eval_loss.item()
            # Aktualisierung des States der valid_accuracy
            valid_accuracy.update(pred, label.type(torch.LongTensor))
    
    valid_loss_hist[epoch] = eval_running_loss/num_eval_batches

    # Berechnung der Validierungsgenauigkeit für die Epoche
    total_valid_accuracy = valid_accuracy.compute()
    # Hinzufügen der Validierungsgenauigkeit zum Dictionary
    valid_accuracy_hist[epoch] = total_valid_accuracy.cpu().numpy()

    print(f"Training Accuracy for epoch {epoch}: {total_train_accuracy}")
    print(f"Validation Accuracy for epoch {epoch}: {total_valid_accuracy}")
    train_accuracy.reset()
    valid_accuracy.reset()

Training Accuracy for epoch 0: 0.0
Validation Accuracy for epoch 0: 0.0
Training Accuracy for epoch 1: 0.0
Validation Accuracy for epoch 1: 0.0
Training Accuracy for epoch 2: 0.0
Validation Accuracy for epoch 2: 0.0
Training Accuracy for epoch 3: 0.0
Validation Accuracy for epoch 3: 0.0
Training Accuracy for epoch 4: 0.0
Validation Accuracy for epoch 4: 0.0
Training Accuracy for epoch 5: 0.0
Validation Accuracy for epoch 5: 0.0
Training Accuracy for epoch 6: 0.0
Validation Accuracy for epoch 6: 0.0
Training Accuracy for epoch 7: 0.0
Validation Accuracy for epoch 7: 0.0
Training Accuracy for epoch 8: 0.0
Validation Accuracy for epoch 8: 0.0
Training Accuracy for epoch 9: 0.0
Validation Accuracy for epoch 9: 0.0
Training Accuracy for epoch 10: 0.0
Validation Accuracy for epoch 10: 0.0
Training Accuracy for epoch 11: 0.0
Validation Accuracy for epoch 11: 0.0
Training Accuracy for epoch 12: 0.0
Validation Accuracy for epoch 12: 0.0
Training Accuracy for epoch 13: 0.0
Validation Accuracy fo

In [63]:
train_loss_df = pd.DataFrame.from_dict(train_loss_hist, orient = "index").reset_index()
train_loss_df["type"] = "Train"
valid_loss_df = pd.DataFrame.from_dict(valid_loss_hist, orient = "index").reset_index()
valid_loss_df["type"] = "Test"
loss_df = pd.concat([train_loss_df,valid_loss_df])
loss_df.columns = ["Epoch", "Loss", "Type"]

In [64]:
train_eval_loss_chart = alt.Chart(loss_df).mark_line().encode(
    x = alt.X('Epoch', title='Anzahl Epochen'),
    y = alt.Y('Loss',title='Mittlerer Fehler'),
    color = alt.Color('Type',title='Typ')
)
train_eval_loss_chart

alt.Chart(...)

In [65]:
valid_accuracy_hist

{0: array(0., dtype=float32),
 1: array(0., dtype=float32),
 2: array(0., dtype=float32),
 3: array(0., dtype=float32),
 4: array(0., dtype=float32),
 5: array(0., dtype=float32),
 6: array(0., dtype=float32),
 7: array(0., dtype=float32),
 8: array(0., dtype=float32),
 9: array(0., dtype=float32),
 10: array(0., dtype=float32),
 11: array(0., dtype=float32),
 12: array(0., dtype=float32),
 13: array(0., dtype=float32),
 14: array(0., dtype=float32),
 15: array(0., dtype=float32),
 16: array(0., dtype=float32),
 17: array(0., dtype=float32),
 18: array(0., dtype=float32),
 19: array(0., dtype=float32),
 20: array(0., dtype=float32),
 21: array(0., dtype=float32),
 22: array(0., dtype=float32),
 23: array(0., dtype=float32),
 24: array(0., dtype=float32),
 25: array(0.00166806, dtype=float32),
 26: array(0.01834862, dtype=float32),
 27: array(0.07172644, dtype=float32),
 28: array(0.15262718, dtype=float32),
 29: array(0.22852378, dtype=float32),
 30: array(0.27189323, dtype=float32),
 

In [66]:
train_accuracy_hist

{0: array(0., dtype=float32),
 1: array(0., dtype=float32),
 2: array(0., dtype=float32),
 3: array(0., dtype=float32),
 4: array(0., dtype=float32),
 5: array(0., dtype=float32),
 6: array(0., dtype=float32),
 7: array(0., dtype=float32),
 8: array(0., dtype=float32),
 9: array(0., dtype=float32),
 10: array(0., dtype=float32),
 11: array(0., dtype=float32),
 12: array(0., dtype=float32),
 13: array(0., dtype=float32),
 14: array(0., dtype=float32),
 15: array(0., dtype=float32),
 16: array(0., dtype=float32),
 17: array(0., dtype=float32),
 18: array(0., dtype=float32),
 19: array(0., dtype=float32),
 20: array(0., dtype=float32),
 21: array(0., dtype=float32),
 22: array(0., dtype=float32),
 23: array(0., dtype=float32),
 24: array(0., dtype=float32),
 25: array(0.00107219, dtype=float32),
 26: array(0.00929235, dtype=float32),
 27: array(0.03502502, dtype=float32),
 28: array(0.11007863, dtype=float32),
 29: array(0.19442458, dtype=float32),
 30: array(0.268406, dtype=float32),
 31

In [67]:
# Erste Epoche löschen, da diese nicht neu initialisiert wird und Daten aus vorherigen Schritten enthält

del train_accuracy_hist[0]
del valid_accuracy_hist[0]

In [68]:
train_accuracy_df = pd.DataFrame.from_dict(train_accuracy_hist, orient = "index").reset_index()
train_accuracy_df["type"] = "Train"
valid_accuracy_df = pd.DataFrame.from_dict(valid_accuracy_hist, orient = "index").reset_index()
valid_accuracy_df["type"] = "Test"
accuracy_df = pd.concat([train_accuracy_df,valid_accuracy_df])
accuracy_df.columns = ["Epoch", "Accuracy", "Type"]

In [69]:
accuracy_df

,Epoch,Accuracy,Type
0,1,0.000000,Train
1,2,0.000000,Train
2,3,0.000000,Train
3,4,0.000000,Train
4,5,0.000000,Train
...,...,...,...
44,45,0.309425,Test
45,46,0.309425,Test
46,47,0.309425,Test
47,48,0.309425,Test


In [70]:
train_eval_accuracy_chart = alt.Chart(accuracy_df).mark_line().encode(
    x = alt.X('Epoch',title='Anzahl Epochen'),
    y = alt.Y('Accuracy',title='Genauigkeit'),
    color = alt.Color('Type',title='Typ')
)
train_eval_accuracy_chart

alt.Chart(...)